In [1]:
import xarray as xr
import numpy as np
from time import time
import glob
#from tqdm.contrib.concurrent import process_map 
#import multiprocessing as mp
import os
import PyCO2SYS as pyco2
from netCDF4 import Dataset


In [2]:
def calc_omega_a( alkalinity, dic, temperature, salinity, pressure ):
    results = pyco2.sys(
        par1 = alkalinity,
        par1_type = 1,
        par2 = dic,
        par2_type = 2,
        temperature = temperature,
        salinity = salinity,
        pressure = pressure
    )
    return results['saturation_aragonite']


In [3]:
path0='/home/amh001/space_fs7/DATA/NEP36-I/HINDCAST/inputs_714x1020/NEP36-channels_mask.nc'
ncT= Dataset(path0,'r')
mask_chan = ncT.variables['mask'][:,:,:]


In [4]:
datapath = '/home/amh001/space_fs7/DATA/NEP36-CanOE/Benthic_2023/'


x_slice = slice(215,713)
y_slice = slice(180,750)

In [5]:
def getdata(YEAR,mD):
    datapath = '/home/amh001/space_fs7/DATA/NEP36-CanOE/'
    file = f"{datapath}/Benthic_2023/NEP36-CanOE_1d_benthic_shelf_"+str(YEAR)+"*.nc"
    ds = xr.open_mfdataset(file, decode_times=False, parallel=True)
    ds = ds.isel(x=x_slice, y=y_slice)
    D=ds['depths'].values
    O2=ds['O2'].values
    T=ds['T'].values
    DIC=ds['DIC'].values
    ALK=ds['ALK'].values
    S=ds['S'].values

    mask=(0<=D[0,:,:])&(D[0,:,:]<mD)
    mask=(mask_chan[0,y_slice,x_slice]>0)&mask
    print('calculating omega ...')
    omega_A=calc_omega_a( ALK[:,mask], DIC[:,mask],T[:,mask],S[:,mask],D[:,mask] )
    print('done')
    O2=O2[:,mask]
    T=T[:,mask]
    time=ds['time'].values
    nav_lon=ds['nav_lon'].values[mask]
    nav_lat=ds['nav_lat'].values[mask]
    D=D[:,mask]
    ds2 = xr.Dataset(
    data_vars=dict(
        T=(["time",  "x"], T),
        O2=(["time",  "x"], O2),
        Omega_A=(["time",  "x"], omega_A),
        depths=(["time", "x"], D),
    ),
    coords=dict(
        nav_lon=([ "x"], nav_lon),
        nav_lat=([ "x"], nav_lat),
        time=time,
    ),
    attrs=dict(description="cluster of depths less than "+str(mD)),
 )
    ds2.time.attrs['units'] = "seconds since 1900-01-01 00:00:00"
    ds2.T.attrs['units'] = "C"
    ds2.O2.attrs['units'] = "mmol/m3"
    ds2.Omega_A.attrs['units'] = ""

    ds2.depths.attrs['units'] = "m"
    ds2.nav_lat.attrs['units'] = "degrees_north"
    ds2.nav_lon.attrs['units'] = "degrees_east"
    
    
    return ds2

In [6]:
mD=69
for YEAR in np.arange(1996,1997):
#YEAR=1996  
    ds2=getdata(YEAR,mD)

    outfile="/home/amh001/space_fs7/DATA/NEP36-CanOE/cluster_depth/NEP36_cluster_less"+str(mD)+"m_y"+str(YEAR)+".nc"

    print('saving ', outfile)
    
    
    #        ds = xr.open_dataset(filename0)
    #       z_index=xr.DataArray(nansum(tmask, axis=0)-1) # Replace this with your mkbt variable
    comp = dict(zlib=True, complevel=1)
    encoding = {var: comp for var in ds2.data_vars}
    ds2.to_netcdf(outfile,unlimited_dims='time', encoding=encoding)


calculating omega ...
done
saving  /home/amh001/space_fs7/DATA/NEP36-CanOE/cluster_depth/NEP36_cluster_less69m_y1996.nc
